In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

from pof.failure_mode import FailureMode
from interface.layouts import *

In [2]:
fm = FailureMode().set_default()

In [3]:
from interface.layouts import *

# Failure Mode Tests

mfml = make_failure_mode_layout(fm, prefix="fm")

# Task Tests
task = fm.tasks['ocr']
mtl = make_task_layout(task)

# Trigger Tests
triggers= fm.tasks['ocr'].triggers()
mttl = make_task_trigger_layout(triggers)

condition_triggers = fm.tasks['ocr'].triggers()['condition']
mctf = make_condition_trigger_form(condition_triggers)


# Impact Tests

impacts= fm.tasks['ocr'].impacts()
mtil = make_task_impact_layout(impacts)

state_impacts = fm.tasks['ocr'].impacts()['state']
msil = make_state_impact_layout(state_impacts)

condition_impacts = fm.tasks['ocr'].impacts()['condition']
mcil = make_condition_impact_layout(condition_impacts)

condition_impact = fm.tasks['ocr'].impacts()['condition']['wall_thickness']
mtif = make_condition_impact_form(condition_impact)

In [137]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = html.Div([
    html.Div(children='dummy text', id='test_output'),
    mfml
])

collapses = []
for fm_name, fm in {'fm': fm}.items():
    collapses = collapses + [fm_name]
    for task in fm.tasks:
        collapses = collapses + [fm_name + '-' + task]

@app.callback(
    #Output("test_output", "children"),
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapses],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapses],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapses],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapses:

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())
    
    return is_open

#Execute
app.run_server(mode='inline')

In [73]:
collapses = ['fm', 'cm']
[Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapses]

[<Input `fm-collapse-button.n_clicks`>]

In [75]:
collapses = ['fm', 'cm']
[Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapses]

[<Input `fm-collapse-button.n_clicks`>, <Input `cm-collapse-button.n_clicks`>]

In [128]:
collapses

['fm', 'fm-inspection', 'fm-ocr', 'fm-cm']